# Data Wrangling: Join, Combine, and Reshape

In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Hierarchical Indexing

allows you to have multiple index levels (think year - months). Good for working with 'higher dimensional data in a lower dimensional form.'

In [4]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

# the above gets unpacked to the below output.

data

a  1    1.246435
   2    1.007189
   3   -1.296221
b  1    0.274992
   3    0.228913
c  1    1.352917
   2    0.886429
d  2   -2.001637
   3   -0.371843
dtype: float64

In [5]:
# now when calling .index, it shows we have a MultiIndex
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [11]:
# calling the outer index returns all values of the inner indexes.
data['b']
data['b':'c']
data.loc[['b', 'd']]

b  1    0.274992
   3    0.228913
d  2   -2.001637
   3   -0.371843
dtype: float64

In [14]:
# the below returns all outer indexes, but only inner index of [2]. Remember, commas for accessing multi-levels.

data.loc[:, 2]

a    1.007189
c    0.886429
d   -2.001637
dtype: float64

In [16]:
# Hierarchical indexing plays an important role in reshaping data and group-based operations like forming 
# a pivot table. 
# using .unstack() will reshape our data.

data.unstack()

,1,2,3
a,1.246435,1.007189,-1.296221
b,0.274992,NaN,0.228913
c,1.352917,0.886429,NaN
d,NaN,-2.001637,-0.371843


In [17]:
# the opposite of .unstack() is .stack(), this can return or more our data back to the hierarchical indexing.

data.unstack().stack()

a  1    1.246435
   2    1.007189
   3   -1.296221
b  1    0.274992
   3    0.228913
c  1    1.352917
   2    0.886429
d  2   -2.001637
   3   -0.371843
dtype: float64

In [19]:
# with a DataFrame, we can have a hierarchical structure for both the indexes AND the columns;

frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [20]:
# just like we can use df.index.name = [] to name an entire index column, we can use .index.names = [ , ] to 
# name our hierarchical indices. Same goes for columns. This can help our dataframes be more readable.

frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [29]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

### Reordering and Sorting Levels

Sometimes you will need to rearrange the order of the levels on an axis or sort the data by values in a specific level. the .swaplevel() takes two level numbers or names and returns the object with those two swapped. Data is unaltered.

In [30]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [33]:
# .sort_index() sorts only using one level of the hierarchy. 
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [35]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and Series have a level option.
We can find statistical valuations of given levels of our hierarchy. (i.e. let's check sales for August of each year, then check sales for March of each year. If our hierarchy was Year - Month, we wouldn't want the values of entire years for the scenario).

In [37]:
frame.sum(level='key2')
frame.sum(level='color', axis=1)

/var/folders/6k/dfzdrspj2yn7xlvg1wz81x0m0000gp/T/ipykernel_75338/2181692498.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='key2')
/var/folders/6k/dfzdrspj2yn7xlvg1wz81x0m0000gp/T/ipykernel_75338/2181692498.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='color', axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns

In [38]:
# sometimes we will want to move the row index to the DataFrame's column. Let's start with example below;

frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [40]:
# .set_index (['c', 'd']) here moves our columns to a hierarchical index.

frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [43]:
# by default, the columns are removed from the DataFrame and moved to the index. If we'd like to also keep them
# as columns, we can do so with , drop=False.

frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [46]:
# and if we gaffe...  df.reset_index() ! This moves hierarchical indexes to colummns. Index to column.

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Datasets

There are a number of ways to combine objects together with Pandas.

- pandas.merge connects rows in DataFrames based on keys (similar to SQL).
- pandas.concat concatenates or "stacks" objects together along an asix.
- combine_first instance method enables splicing together overlapping data to fill in missing values in one object with values from another ( think back to the dummy table we saw in a previous chapter).

### Database-Style DataFrame Joins

In [47]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [49]:
# merge or 'join' operations combine datasets by linking rows using one or more keys.
# this is central to relational databases (i.e. SQL-=based).
# the Pandas merge function is the main entry point for using these algos on data.

# this example shows a many-to-one join. The data in df1 has multiple rows named 'b'. df2 only has one.

pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [50]:
# by default, .merge uses overlapping column names as keys. to explicitly tell it to do so, we can use on='key'.

pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [51]:
# if column names are different in each object, we can specify them separately w/ left_on & right_on.
# by default this does an inner-join, combining the data where keys match.


df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [53]:
# we can also do an outer join w/ how='outer'. this retains even non-matching column values
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df1
df2
pd.merge(df1, df2, on='key', how='left')

In [54]:
# Table for different join types with how argument on pg. 229

In [55]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [59]:
# for another inner join example

customer=pd.DataFrame({
    'id':[1,2,3,4,5,6,7,8,9],
    'name':['Olivia','Aditya','Cory','Isabell','Dominic','Tyler','Samuel','Daniel','Jeremy'],
    'age':[20,25,15,10,30,65,35,18,23],
    'Product_ID':[101,0,106,0,103,104,0,0,107],
    'Purchased_Product':['Watch','NA','Oil','NA','Shoes','Smartphone','NA','NA','Laptop'],
    'City':['Mumbai','Delhi','Bangalore','Chennai','Chennai','Delhi','Kolkata','Delhi','Mumbai']
})
customer

,id,name,age,Product_ID,Purchased_Product,City
0,1,Olivia,20,101,Watch,Mumbai
1,2,Aditya,25,0,NA,Delhi
2,3,Cory,15,106,Oil,Bangalore
3,4,Isabell,10,0,NA,Chennai
4,5,Dominic,30,103,Shoes,Chennai
5,6,Tyler,65,104,Smartphone,Delhi
6,7,Samuel,35,0,NA,Kolkata
7,8,Daniel,18,0,NA,Delhi
8,9,Jeremy,23,107,Laptop,Mumbai


In [60]:
# if we had a product table, we could use this to merge product & customer tables at product_ID.
# showing which products were sold and to which customers.

# pd.merge(product,customer,on='Product_ID')

NameError: name 'product' is not defined

In [56]:
# to merge with multiple keys, pass a list of column names;

left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [61]:
# what if our column names overlap? And maybe shouldn't? 
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [62]:
# we can add suffixes to the columns to see where the original values came from and avoid conflicting columns.
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


pg. 232 has a table of merge function arguments (left, right, how, on, left_on, right_on, left_index, right_index, sort, suffixes, copy, indicator).

### Merging on Index

In [63]:
# in some cases, the merge key will be found in the index. When want to merge with the index as a key
# we can use left_index or right_index as the joining argument.

left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
left1
right1
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [65]:
# the default merge method is to intersect the join keys, we can form the union of the two with an outer join.

pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [66]:
# hierarchically organized data makes things a little more complicated, because joining on index is implicitly
# a multiple-key merge..

lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
lefth
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [71]:
# to get around this, we can set left_on / right_on to a list of indexes. We list multiple columns to merge on.

pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [72]:
# using indexes of both sides is also possible.

left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
left2
right2
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [70]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [73]:
# DataFrame's .join method perfroms a left join on the join keys, preserving the left frame's row index.

left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [74]:
# for simple index-on index merges, a list of DataFrames can be joined. [right2, another], and arguments can still
# passed.

another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another
left2.join([right2, another])
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating Along an Axis

a.k.a. 'binding' or 'stacking'

In [95]:
# NumPy has .concatenate which takes a ([base, add], arguments)
# axis by default is 0, this would have returned arr with its original form but repeated.
# axis=1 allowed adding to the end of each item in arr.

arr = np.arange(12).reshape((3, 4))
arr
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [96]:
# Pandas also has a concatenate method, which takes a list to concatenate.
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [97]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [98]:
# by setting axis=1 we can make more of a traditional table. where indexes don't match values will be NaN

pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [99]:

s4 = pd.concat([s1, s3])
s4
# pd.concat([s1, s4], axis=1)
pd.concat([s1, s4], axis=1, join='inner')

# notice the 'f' and 'g' labels disappeared when using join='inner'

,0,1
a,0,0
b,1,1


In [102]:
# you can specify the axes to be used on the other axes with join_axes;
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [107]:
# a potential issue of concatenation is the results being unidentifiable in the result, in regards to where
# they originally came from.
# you could create a hierarchical index on the concatenation axis by using the 'keys' argument.

result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result
# result.unstack()

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [108]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [110]:
# when concatenating along axis=1, the keys become the DataFrame column headers.

pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [113]:
# the same logic for combining series applies to DataFrame objects

df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df1
df2
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

# our keys became our hierarchical column values

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [114]:
# when passing in a dict, the dict's keys will be used for the keys option;

pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [115]:
# as usual, we can give names to these columns / indexes

pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [116]:
# the last case is when the row index doesn't have relevant data.

df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1
df2

,b,d,a
0,-0.713544,-0.831154,-2.370232
1,-1.860761,-0.860757,0.560145


In [118]:
# if we don't need to retain the index, we can ignore_index=True while concatenating.

pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.669025,-0.438570,-0.539741,0.476985
1,3.248944,-1.021228,-0.577087,0.124121
2,0.302614,0.523772,0.000940,1.343810
3,-2.370232,-0.713544,NaN,-0.831154
4,0.560145,-1.860761,NaN,-0.860757


#### pg 240 has concat function arguments (objs, axis, join, join_axes, keys, levels, names, verify_integrity, ignore_index)

### Combining Data with Overlap

In some situations there may be an overlap in indexes in part or in full.
Below we can see such a situation, where index values match but values per row don't necessarily match.

In [123]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
a
b
np.where(pd.isnull(a), b, a)

# np.where can perform an array-oriented if/else.
# for this example, if an index in series 'a' is null, fill it with 'b's value. Otherwise, keep 'a's value.

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [124]:
# combine_first() can be thought of as 'patching' missing data in the calling object with data from the passed
# object. 
# here, data from Series 'a' gets patched to Series 'b'

b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [125]:
# Pandas combine_first does the same, but column by column.

df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## Reshaping and Pivoting

Rearranging tabular data is also called 'reshape' or 'pivot' operations.

### Reshaping with Hierarchical Indexing

In [126]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [131]:
# .stack() rotates our data. The columns are pivoted to rows, producing a series.

result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [130]:
# unstack does the inverse.

result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [133]:
# by default, the innermost level is stacked. we can stack different levels by passing a level number or name;

result.unstack(0)
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [135]:
# Unstacking might introduce missing data if all of the values in the level aren't found in each subgroup.

s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2
data2.unstack()

# s2 does not have values for 'a' and 'b', and s1 does not have a value for 'e'

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [138]:
# stacking filters out missing data by default. we can retain these missing values with dropna=False
data2.unstack()
data2.unstack().stack()
# data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [139]:
# when unstacking a DataFrame, the unstacked level becomes the lowest level in the result.

df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [141]:
# when calling a stack, we can indicate the name of the axis to stack, just like we can w/ axis=0/1.
# here we unstack state (moving it to columns) and stack side (now including it in our series).
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting “Long” to “Wide” Format

A common way to store multiple time series in databases & CSV is in the 'long' or 'stacked' format.

In [145]:
data = pd.read_csv('examples/macrodata.csv')
data.head()
# pd.PeriodIndex combines year & quarter columns in this example.
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [147]:
# our data, stacked according to realgdp as 'value' along axis 0.
ldata[:10]

# now we see each column value in a series according to the shared time stamp.
# this is the 'long' format. It can be used with single or multiple keys. Our keys here are each of the
# time stamps & each of the columns (infl, realgdp, unemp).

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [149]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted 

# but sometimes we'll want those columns to line up according to their shared index.
# the pivot method can tell us w/ keys to pivot (move from being row values to columns).

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [151]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

# just adding a column to the dataframe for the next example

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.375498
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.958704
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.794336
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-1.605108
4,1959-06-30 23:59:59.999999999,infl,2.340,0.543710
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.925166
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-1.469629
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.399592
8,1959-09-30 23:59:59.999999999,unemp,5.300,1.417343
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-0.897609


In [153]:
# suppose there were two columsn we wanted to pivot at once.
# .pivot() can take multiple arguments.
# in the above example we also passed 'value' to pivot. In this example we only pass the two
# and it results in a hierarchical columns.

pivoted = ldata.pivot('date', 'item')
pivoted[:5]
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [154]:
# pivot is equivalent to creating a hierarchical index using set_index followed by a call to unstack.

unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.958704 -0.375498   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.543710 -1.605108   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.399592 -1.469629   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.844805 -0.897609   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.027734 -1.490932   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.667880 -0.025208   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  1.287155  0.210689   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.794336  
1959-06-30 23:59:59.999999999  0.925166  
1959-09-30 23:59:59.999999999  1.417343  
1959-12-31 23:59:59.999999999  1.253168  
1960-03-31 23:59:59.999999999  1.375236  
1960-06-30 23:59:59.999999999 -2.868018  
1960-09-30 23:59:59.999999999 -0.574306

### Pivoting “Wide” to “Long” Format

- The inverse of pivoting is an operation called pandas.melt .
- Rather than transforming one column into many in a new DataFrame, it merges multiple columns in to one. 
- The result is a DataFrame that is longer than the input.

In [155]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [159]:
# when melting a table, if there is a grouping we want to retain, we set it in the parameters.
# below we melt df, and retain the 'key' grouping.
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [163]:
# like stacking & unstacking, melt works well with pivot.
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [161]:
# if we want those generic indexes back.
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [164]:
# .melt() can be used without any group identifiers, too;
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [165]:
pd.melt(df, value_vars=['A', 'B', 'C'])
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


## Conclusion